In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# prompt: extract the zip file

import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/English_data.zip', 'r') as zip_ref:
  zip_ref.extractall('/content')

In [4]:
# Constants
BATCH_SIZE = 32
IMAGE_SIZE = 224
CHANNELS = 3
EPOCHS = 50
TEST_SIZE=0.2
RANDOM_STATE=24
PATIENCE=20
LR_PATIENCE=20
FACTOR_LR=0.5

In [5]:
# Load pre-trained MobileNetV2 model without the top classification layer
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

9406464/9406464 [==============================] - 2s 0us/step


In [6]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(315, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
from tensorflow.keras.optimizers import Adam
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory("English_data",shuffle=True,
                                                            image_size=(IMAGE_SIZE,IMAGE_SIZE),
                                                            batch_size=BATCH_SIZE,
                                                            validation_split=TEST_SIZE,
                                                            subset='training',
                                                            seed=RANDOM_STATE)
test_ds=tf.keras.preprocessing.image_dataset_from_directory("English_data",shuffle=True,
                                                            image_size=(IMAGE_SIZE,IMAGE_SIZE),
                                                            batch_size=BATCH_SIZE,
                                                            validation_split=TEST_SIZE,
                                                            subset='validation',
                                                            seed=RANDOM_STATE)

Found 7525 files belonging to 315 classes.
Using 6020 files for training.
Found 7525 files belonging to 315 classes.
Using 1505 files for validation.


In [9]:
train_ds=train_ds.map(lambda x,y:(x/255.0,tf.one_hot(y,315)))
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.map(lambda x,y:(x/255.0,tf.one_hot(y,315)))
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [10]:

# Train the model
model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=test_ds,verbose=1,batch_size=BATCH_SIZE)

# Optionally, unfreeze some layers and continue training
'''for layer in base_model.layers[:100]:
    layer.trainable = True

# Recompile the model (necessary after changing trainable attributes)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training
model.fit(
    train_generator,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs_fine_tuning,
    validation_data=validation_generator,
    validation_steps=total_val // batch_size)'''


Epoch 1/50
189/189 [==============================] - 26s 73ms/step - loss: 5.1074 - accuracy: 0.0865 - val_loss: 4.4073 - val_accuracy: 0.1342
Epoch 2/50
189/189 [==============================] - 10s 52ms/step - loss: 3.5487 - accuracy: 0.2542 - val_loss: 3.3033 - val_accuracy: 0.2777
Epoch 3/50
189/189 [==============================] - 10s 52ms/step - loss: 2.5649 - accuracy: 0.4480 - val_loss: 2.6544 - val_accuracy: 0.3867
Epoch 4/50
189/189 [==============================] - 10s 51ms/step - loss: 1.9675 - accuracy: 0.5824 - val_loss: 2.2722 - val_accuracy: 0.4478
Epoch 5/50
189/189 [==============================] - 10s 52ms/step - loss: 1.5701 - accuracy: 0.6746 - val_loss: 1.9982 - val_accuracy: 0.4950
Epoch 6/50
189/189 [==============================] - 10s 52ms/step - loss: 1.2839 - accuracy: 0.7447 - val_loss: 1.8122 - val_accuracy: 0.5415
Epoch 7/50
189/189 [==============================] - 10s 55ms/step - loss: 1.0767 - accuracy: 0.7942 - val_loss: 1.6758 - val_accuracy:

"for layer in base_model.layers[:100]:\n    layer.trainable = True\n\n# Recompile the model (necessary after changing trainable attributes)\nmodel.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),\n              loss='categorical_crossentropy',\n              metrics=['accuracy'])\n\n# Continue training\nmodel.fit(\n    train_generator,\n    steps_per_epoch=total_train // batch_size,\n    epochs=epochs_fine_tuning,\n    validation_data=validation_generator,\n    validation_steps=total_val // batch_size)"

In [11]:
model.save('/content/drive/MyDrive/MobileNet_71.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
